In [3]:
import platform
import warnings
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from matplotlib import rc
from bert_sim import word_sim
from gensim.models import Word2Vec

warnings.filterwarnings(action='ignore')


HTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/kykim/bert-kor-base/resolve/main/vocab.txt

In [9]:
class check_data():
    def __init__(self,df):
        self.df = df
        #self.wiki_model = Word2Vec.load('../ko/ko.bin')
        if platform.system() == 'Windows':
            self.font = 'Malgun Gothic'
        else:
            self.font = 'NanumGothic'
        self.df['효능_Morps'] = self.df['효능'].map(lambda x: self.getMorps(x))
        self.df['효능_NV'] = self.df['효능_Morps'].map(lambda x: self.getNV(x))
        self.df['효능_N'] = self.df['효능_Morps'].map(lambda x: self.getN(x))
  
    def topn(self,n):
        self.df['효능_count_topn_NV']  = self.df['효능_NV'].map(lambda x: self.counter(x,n)) 
        self.df['효능_count_topn_N']  = self.df['효능_N'].map(lambda x: self.counter(x,n)) 

    def result(self,idx, top , dis):
        energy = ['정열', '활기', '기력', '생기', '활발','--']
        recovery = ['낫' '치료', '휴식' ,'극복' , '재활', '--']
        cycle = ['돌', '유지', '조절' , '공급' ,'생명', '--']
        pury = ['진정' , '맑' , '없애' , '깨끗', '해소', '--']        
        effs_dims_name = ['에너지', '회복', '순환', '정화']
        effs_dims =  [energy, recovery, cycle , pury]
        sNV = self.df['효능_NV'][idx]
        
        result = []
        for dim in effs_dims:
            ab = []
            for sim_word in dim:
                sub = []
                for word in sNV:
                    try:
                        sub.append(word_sim(sim_word , word))
                    except:
                        sub.append(0)
                ab.append(sub)
                aab = np.array(ab).flatten().tolist()
            aab = np.array(sorted(aab,reverse=True))[:top].mean()
            result.append(aab)
        
        if dis == True:
            from matplotlib import rc
            plt.rc('font', family=self.font)

            word = self.df.iloc[idx][0]
            std = np.std(result)
            
            plt.figure()
            plt.bar(np.arange(len(result)),result)
            plt.title(f'{word}\'s mapping for top {top} using 유사어')
            plt.xticks(np.arange(len(effs_dims_name)), effs_dims_name)
            plt.yticks(np.arange(0,1.2,0.2))
            plt.show()
#            print(f'{word}s std is {std}' )
        return result
        
    def getMorps(self,sent):
        return Okt().pos(sent , norm=False , stem = True) 

    def getNV(self, morps):
        stop_words = "하다 것 등 수 그 이 더욱 대해 예 머 매 내 수도 무엇 모든 이다 때 곧 식 또한 좀 꼭 번 해 과 바로 더욱 논 보이 난 "
        stop_words = set(stop_words.split(' '))

        result = []
        for item , morp in morps:
            if morp == 'Verb' or morp == 'Noun': 
                result.append(item)
        s_result = []
        
        for word in result:
            if word not in stop_words:
                s_result.append(word)
        return s_result
    
    def getN(self, morps):
        stop_words = "하다 것 등 수 그 이 더욱 대해 예 머 매 내 수도 무엇 모든 이다 때 곧 식 또한 좀 꼭 번 해 과 바로 더욱 논 보이 난 "

        stop_words = set(stop_words.split(' '))

        result = []
        for item , morp in morps:
            if morp == 'Noun': 
                result.append(item)
        s_result = []
        
        for word in result:
            if word not in stop_words:
                s_result.append(word)
        s_result = np.unique(s_result)
        return s_result

    def counter(self,sents, n):
        return sorted(collections.Counter(sents),reverse=True)[0:n]

    def get_heatmap(self, emotion_idx,figsize):
        energy = ['정열', '활기', '기력', '생기', '활발','--']
        recovery = ['낫', '치료', '휴식' ,'극복' , '재활', '--']
        cycle = ['운반', '유지', '조절' , '공급' ,'생명', '--']
        pury = ['진정' , '맑' , '없애' , '깨끗하', '해소', '--']        
        all_dims = energy + recovery + cycle + pury
        
        plt.rc('font', family=self.font)
        sents = self.df['효능'][emotion_idx]
        print(sents)
        
        result = []
        for word in sents:
            sub = []
            for dim_word in all_dims:
                try:
                    if dim_word == '--':
                        sub.append(0)
                        continue
                    sub.append(word_sim(dim_word , word))
                except:
                    sub.append(0)
            result.append(sub)

        fig = plt.figure(figsize=figsize)
        fig.set_facecolor('white')
        em_name = self.df['감정'][emotion_idx]
        plt.title(f'{em_name}에서 단어들간의 유사도 확인')
        sns.heatmap(result,annot=True)
        plt.xticks(np.arange(len(all_dims)),all_dims)
        plt.yticks(np.arange(len(sents)),sents)
        plt.show()
              


In [5]:
# 감정 데이터 확인
em_data = pd.read_csv("../data/emotion_data_hand_craft.csv")
em_data

,감정,효능
0,행복,향유 낙관적 느낌 생각 즐기
1,우울,행동 도움 생활 치료 명상
2,슬픔,사랑 도움 가족 시간 활동
3,열정,상기 집중 노력 몰입 휴식
4,분노,정지 회피 긍정 마음 호흡
5,두려움,직면 변화 대화 집중 용기


### 각 차원의 유사어와 각 감정의 키워드 사이의 유사도를 확인
bert 모델 사용

In [10]:
hXYZ = check_data(em_data)
for i in em_data.index:
    hXYZ.get_heatmap(i, (20,4))

향유 낙관적 느낌 생각 즐기
행동 도움 생활 치료 명상
사랑 도움 가족 시간 활동
상기 집중 노력 몰입 휴식
정지 회피 긍정 마음 호흡
직면 변화 대화 집중 용기
